# Smartphone's Screen Crack Detection and Scoring using EfficientNet

In [18]:
# ============================================
# 0. IMPORT
# ============================================
import os, json, numpy as np, pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ============================================
# 1. CONFIG
# ============================================
IMG_SIZE   = (224, 224)
BATCH_SIZE = 16

TRAIN_DIR = "data/train"
VAL_DIR   = "data/val"
TEST_DIR  = "data/test"

MODEL_STAGE1_PATH = "effnet_3class_stage1.keras"
MODEL_FINAL_PATH  = "effnet_3class_best.keras"

# Data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.08,
    height_shift_range=0.08,
    brightness_range=[0.85, 1.15],
    zoom_range=0.08,
    horizontal_flip=True,
)
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)
train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print("class_indices (train):", train_gen.class_indices)

# Save class indices to JSON
with open("class_indices_3class.json", "w") as f:
    json.dump(train_gen.class_indices, f)

# ============================================
# 3. CLASS WEIGHTS (KALAU IMBALANCED)
# ============================================
class_weights = None
unique_classes = np.unique(train_gen.classes)
if len(unique_classes) == 3:
    cw = compute_class_weight(
        class_weight="balanced",
        classes=unique_classes,
        y=train_gen.classes
    )
    class_weights = dict(zip(unique_classes, cw))
    print("class_weights:", class_weights)


Found 202 images belonging to 3 classes.


Found 42 images belonging to 3 classes.
Found 73 images belonging to 3 classes.
class_indices (train): {'broken': 0, 'safe': 1, 'warning': 2}
class_weights: {np.int32(0): np.float64(0.8977777777777778), np.int32(1): np.float64(1.9803921568627452), np.int32(2): np.float64(0.7240143369175627)}


In [19]:
from pathlib import Path
from PIL import Image

root = Path("data")  # folder utama dataset kamu
bad_files = []

for img_path in root.rglob("*.jpg"):
  try:
    with Image.open(img_path) as img:
      img.verify()  # cek integritas
  except Exception as e:
    print("BAD:", img_path, "->", e)
    bad_files.append(img_path)

for f in bad_files:
  f.unlink()  # hapus file yang rusak

In [20]:
# Model building
base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    pooling="avg"
)

x = layers.Dense(256, activation="relu")(base.output)
x = layers.Dropout(0.4)(x)
out = layers.Dense(3, activation="softmax", name="predictions")(x)

model = models.Model(inputs=base.input, outputs=out)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_8         │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_4     │ (None, 224, 224,  │          7 │ rescaling_8[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_9         │ (None, 224, 224,  │          0 │ normalization_4[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_9[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,378,278 (16.70 MB)

 Trainable params: 4,336,255 (16.54 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [22]:
# Training Stage 1 – freeze backbone, train head
for layer in base.layers:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

ckpt1 = tf.keras.callbacks.ModelCheckpoint(
    MODEL_STAGE1_PATH,
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

early1 = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

history1 = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[ckpt1, early1]
)

Epoch 1/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3610 - loss: 1.1863 
Epoch 1: val_accuracy improved from None to 0.57143, saving model to effnet_3class_stage1.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.3762 - loss: 1.1743 - val_accuracy: 0.5714 - val_loss: 0.9361
Epoch 2/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7277 - loss: 0.7540
Epoch 2: val_accuracy did not improve from 0.57143
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6535 - loss: 0.8008 - val_accuracy: 0.4762 - val_loss: 0.9496
Epoch 3/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7240 - loss: 0.6488
Epoch 3: val_accuracy did not improve from 0.57143
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.7525 - loss: 0.6372 - val_accuracy: 0.4762 - val_loss: 0.9908
Epoch 4/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7295 - loss: 0.5465
Epoch 4: val_accuracy improved from 0.57143 to 0.64286, saving model to effnet_3class_stage1.keras
13/13 ━━━━━━━━━━━━━━━━━━━

In [23]:
# Training Stage 2 – fine-tune some layers of backbone
N_UNFREEZE = 40  # bisa kamu sesuaikan (20–60)
for layer in base.layers[-N_UNFREEZE:]:
    layer.trainable = True

# opsi: freeze BatchNorm untuk stabilitas
for layer in base.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

ckpt2 = tf.keras.callbacks.ModelCheckpoint(
    MODEL_FINAL_PATH,          # misal "effnet_3class_best.keras"
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

early2 = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True,
    verbose=1
)

history2 = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[ckpt2, early2]
)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 911ms/step - accuracy: 0.8664 - loss: 0.3365
Epoch 1: val_accuracy improved from None to 0.61905, saving model to effnet_3class_best.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.8663 - loss: 0.3347 - val_accuracy: 0.6190 - val_loss: 0.9446
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 918ms/step - accuracy: 0.8671 - loss: 0.3925
Epoch 2: val_accuracy did not improve from 0.61905
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8515 - loss: 0.3712 - val_accuracy: 0.5714 - val_loss: 1.0396
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 975ms/step - accuracy: 0.8731 - loss: 0.2792
Epoch 3: val_accuracy did not improve from 0.61905
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8911 - loss: 0.2668 - val_accuracy: 0.6190 - val_loss: 0.9126
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 959ms/step - accuracy: 0.9314 - loss: 0.2000
Epoch 4: val_accuracy did not improve from 0.61905
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9356 - loss

In [24]:
# Load best model and evaluate on test set
model = tf.keras.models.load_model(MODEL_FINAL_PATH)

test_loss, test_acc = model.evaluate(test_gen, verbose=0)
print(f"\nTest Loss : {test_loss:.4f}")
print(f"Test Acc  : {test_acc*100:.2f}%")

y_true = test_gen.classes
y_prob = model.predict(test_gen, verbose=0)
y_pred = np.argmax(y_prob, axis=1)

labels = list(test_gen.class_indices.keys())   # nama kelas sesuai folder
print("Label order:", labels)

print("\n=== Classification Report (3 kelas) ===")
print(classification_report(y_true, y_pred, target_names=labels, digits=4))

print("\n=== Confusion Matrix (3 kelas) ===")
print(confusion_matrix(y_true, y_pred))


Test Loss : 0.9959
Test Acc  : 58.90%
Label order: ['broken', 'safe', 'warning']

=== Classification Report (3 kelas) ===
              precision    recall  f1-score   support

      broken     0.6500    0.5909    0.6190        22
        safe     1.0000    0.4231    0.5946        26
     warning     0.4524    0.7600    0.5672        25

    accuracy                         0.5890        73
   macro avg     0.7008    0.5913    0.5936        73
weighted avg     0.7070    0.5890    0.5926        73


=== Confusion Matrix (3 kelas) ===
[[13  0  9]
 [ 1 11 14]
 [ 6  0 19]]


In [25]:
# Calculate severity scores and save per-image predictions
rep_map = {"safe": 10, "warning": 50, "broken": 90}
rep_scores = np.array([rep_map[c.lower()] for c in labels], dtype=float)

severity_scores = (y_prob * rep_scores).sum(axis=1)

def score_to_bucket(s: float) -> str:
    if s < 30:
        return "safe"
    elif s < 70:
        return "warning"
    else:
        return "broken"

score_bucket = np.array([score_to_bucket(s) for s in severity_scores])

filenames = test_gen.filenames
true_names = [labels[i] for i in y_true]
pred_names = [labels[i] for i in y_pred]

results_df = pd.DataFrame({
    "filename": filenames,
    "true_label": true_names,
    "pred_label": pred_names,
    "prob_" + labels[0]: y_prob[:, 0],
    "prob_" + labels[1]: y_prob[:, 1],
    "prob_" + labels[2]: y_prob[:, 2],
    "severity_score": severity_scores,
    "score_bucket": score_bucket,
})

results_df.to_csv("test_predictions_3class.csv", index=False)
print("\nSaved per-image predictions to: test_predictions_3class.csv")



Saved per-image predictions to: test_predictions_3class.csv
